In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
pd.options.display.float_format = "{:.4f}".format
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.lines import Line2D
plt.rcParams["figure.figsize"] = (16,10)
plt.style.use("~/.dracula.mplstyle")
import datetime as dt
import quandl
import functools
import time, asyncio

# QUANDL Config
import os, sys
parent_path = os.path.abspath(os.path.join('..'))
if parent_path not in sys.path:
    sys.path.append(parent_path)
from common.config import QUANDL_API_KEY
# If you are not Philip, you don't have common.config unless you make your own.

quandl.ApiConfig.api_key = QUANDL_API_KEY # Replace with your API key if running.
api = os.getenv("QUANDL_KEY")

## Define `GLOBAL CONST` Variables

In [2]:
START_DATE = dt.date(year=2014, month=1, day=1)
END_DATE   = dt.date(year=2021, month=1, day=1)
DATERANGE  = {"gte": START_DATE.strftime("%Y-%m-%d"), 
              "lte": END_DATE.strftime("%Y-%m-%d")}

DE_THRESH = 0.1 # Debt/Market Cap Threshold

DB_TED = "FRED/TEDRATE"
DB_T3M = "FRED/DTB3"

GC_DISCOUNT = 0.01

In [3]:
if "CACHED" in globals():
    print("Not re-defining.")
else:
    print("Defining quandl pull")
    CACHED = True
    @functools.lru_cache(maxsize=16)
    def quandl_get_cached(pull, key, start_date, end_date):
        ret = quandl.get(pull, api_key=key, start_date=start_date, end_date=end_date)
        return ret
    
    @functools.lru_cache(maxsize=16)
    def quandl_get_table_cached(db, assets, key, start_date, end_date):
        ret = quandl.get_table(db, ticker=list(assets), api_key=key,
                               date={"gte":START_DATE, "lte":END_DATE})
        return ret

Defining quandl pull


# 1 Understand Your Data

Read all documentation webpages for Zacks Fundamentals B. You will see they supply 6 related tables, FC, FR, MT, MKTV, SHRC and HDM 1. The strategy coding for this assignment will be reasonably easy. The data assembly, deliberately, is the difficult part.

# 2 Define the Universe

Choose at least 200 tickers of US equities such that3 they satisfy the following:

In [4]:
ted_raw = quandl_get_cached(DB_TED, QUANDL_API_KEY, START_DATE-pd.DateOffset(months=3), END_DATE)
t3m_raw = quandl_get_cached(DB_T3M, QUANDL_API_KEY, START_DATE-pd.DateOffset(months=3), END_DATE)

rates = (ted_raw.merge(t3m_raw, on="Date").sum(axis=1) / 100).to_frame("libor")
rates["gc"] = rates["libor"] - GC_DISCOUNT # General Collateral Rate

In [5]:
zmt = quandl.get_table("ZACKS/MT", paginate=True)
active_tickers = zmt[zmt["active_ticker_flag"]=='Y']["m_ticker"].unique()

• debt/market cap ratio is greater than 0.1 somewhere in the period Jan 2014 through Jan 2021 (preferably more than fleetingly)

In [6]:
def filter_debt_mkt_cap(df):
    """Get Summary statistics for debt/market cap for exclusion"""
    mu = df["tot_debt_tot_equity"].mean()
    sig = df["tot_debt_tot_equity"].std()
    mi = df["tot_debt_tot_equity"].min()
    ma = df["tot_debt_tot_equity"].max()
    st = df["per_end_date"].min()
    ed = df["per_end_date"].max()
    tick = df["ticker"].iloc[0]
    return pd.Series([tick, mu, sig, mi, ma, st, ed], index=["ticker", "mean", "std", "min", "max", "start", "end"])

filter_fr = {"columns":["m_ticker", "ticker", "per_end_date", "tot_debt_tot_equity", 
                        "ret_invst",]}
zfr = quandl.get_table("ZACKS/FR",  per_end_date=DATERANGE, qopts=filter_fr, paginate=True)

debt_mkt_cap_df = zfr.dropna().groupby("m_ticker").apply(filter_debt_mkt_cap).reset_index()
allowed_debt_mkt_cap_tickers = debt_mkt_cap_df[debt_mkt_cap_df["min"]>DE_THRESH]["m_ticker"].unique()

• not in the automotive, financial or insurance sector , over the entire period Jan 2014 through Jan 2021

In [7]:
def filter_excluded_sectors(df):
    """Not in automotive, financial or insurance sector
    Auto=5; Finance, Insurance=13"""
    if   (df["zacks_sector_code"]==5).any():  return False
    elif (df["zacks_sector_code"]==13).any(): return False
    else: return True
    
filter_fc = {"columns":["m_ticker", "ticker", "per_end_date", "zacks_sector_code",]}
zfc = quandl.get_table("ZACKS/FC",  per_end_date=DATERANGE, qopts=filter_fc, paginate=True)

sector_df = zfc.groupby("m_ticker").apply(filter_excluded_sectors).to_frame("allowed_sector").reset_index()
allowed_sector_tickers = sector_df[sector_df["allowed_sector"]]["m_ticker"].unique()

• end-of-day adjusted closing prices are available, over the entire period Jan 2014 through Jan 2021

In [11]:
len(set(allowed_debt_mkt_cap_tickers).intersection(set(allowed_sector_tickers), set(active_tickers)))

2536

• has feasible calculation of the ratios specified below , over the entire period Jan 2014 through Jan 2021, including for at least one PER END DATE no more than one year old. Debt ratio of zero is OK.

• debt to market cap

• return on investment

• price to earnings

In [8]:
# Demonstrating the per_end that causes errors

start = time.time()

out = quandl.get_table("ZACKS/FC", per_end_date="2020-09-30",
                       paginate=True)
el = time.time()-start
print(f"Dates Elapsed: {el:.3f}.")

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 220-01-10 00:00:00

# 3 Select Financial Ratios

For this assignment, we will work with the following ratios:

• debt to market cap

• return on investment

• price to earnings

Note that these data items are reported (at best) quarterly. Use annual numbers only when quarterly ones do not exist. As the equity price changes day-to-day, each ratio changes accordingly9, so ultimately the time series you have will be on daily data10. Recall that we did not know any of these numbers until the FC/FILING DATE .

# 4 Analysis

Study performance of weekly or monthly quantile trading strategies using each of these single ratios as well as your choice of least one nontrivial combination of them.

Set initial capital to be 10 times the gross notional of your first month’s set of positions. You may assume zero trading costs, that trading fractional shares and arbitrary positions sizes are possible, that all securities are easy to borrow with a repo rate equal to your funding rate minus 100bp, and that the portfolio capital is equal to the initial capital, adjusted for all realized and unrealized PL to date. Choose either a constant funding rate, or rolling 3-month LIBOR.

Analyze performance of a top-and-bottom decile trading strategy. Now rank based on changes in your ratios rather than the ratios themselves. Play with the effects of sizing positions by rank.